In [1]:
from gensim.models import word2vec
import multiprocessing
from tqdm import tqdm
import pandas as pd
import numpy as np

### Исходный датасет

2 файла:
- data/user_item_interaction.csv колонки: user, item (band_id)
- data/track_meta_processed.tsv колонки song_id, song_name, band, band_preprocessed, band_id

Для обучения word2vec модели рекомендуется использовать band_preprocessed, там содержатся нормализованные названия исполнителей, так с моделью будет проще работать.

Чтобы обучить модель, на вход ей нужно подать файл в формате:
- один пользователь одна строка
- band_id расположены в хронологическом порядке (генератор их сам подменит на нормализованное название) и разделены табуляцией ('\t')

data/band_lists_preprocessed_small.txt - пример, как будет выглядет итоговый файл, на котором можно обучить модель, на нем можно попробовать обучить модель уже сейчас, он содержет лишь 50000 пользователей, после выполнения домашнего задания у вас будет аналогичный файл с 900000 пользователей и качество модели станет заметно лучше

### В качестве домашнего задания нужно будет написать функцию для преобразование датасета:
- когда весь датасет помещается в памяти компьютера
- когда датасет не помещается в памяти (скажем у вас есть 500МБ оперативной памяти)

Последовательность айтемов должна быть сохранена, последовательность пользовательских строк в файле не важна.

Метрика: время преобразования (чем быстрее, тем лучше)
Для честности все варианты мы отскорим на одинаковых мощностях.

In [2]:
!ls -lht data/

total 6205416
-rw-r--r--  1 antonina.goryacheva  staff   641M May  8 08:32 band_lists_preprocessed2.csv
-rw-r--r--@ 1 antonina.goryacheva  staff   641M May  8 08:32 band_lists_preprocessed.csv
-rw-r--r--@ 1 antonina.goryacheva  staff   1.2G May  2 12:42 user_item_interaction.csv
-rw-r--r--@ 1 antonina.goryacheva  staff   464M May  2 12:40 track_meta_processed.tsv
-rw-r--r--@ 1 antonina.goryacheva  staff    19M May  2 12:38 band_lists_preprocessed_small.txt


In [3]:
input_path = 'data/user_item_interaction.csv'
output_path = 'data/band_lists_preprocessed.csv'
output_path2 = 'data/band_lists_preprocessed2.csv'

meta_path = 'data/track_meta_processed.tsv'

w2v_dataset_path = 'data/band_lists_preprocessed_small.txt'

item_type = 'track_id'

In [4]:
pd.read_csv(meta_path, nrows=3, sep='\t')

,song_id,song_name,band,band_processed,band_id
0,0,I Don’t Know Why,Imagine Dragons,imagine dragons,0
1,5,Jingle Bells (Instrumental),Jingle Punks,jingle punks,1
2,6,Dance of the Sugar Plum Fairy,Kevin Macleod,kevin macleod,2


In [5]:
pd.read_csv(input_path, nrows=3)

,user,item
0,0,0
1,1,1
2,1,2


In [6]:
pd.read_csv(w2v_dataset_path, nrows=3, header=None)

,0
0,455748
1,152114\t11597\t38783\t58245\t8804\t5271\t22556...
2,237426\t4067\t20239\t206135\t2630\t1405170\t22396


In [18]:
### пример наивного решения:

# %%time

# df = pd.read_csv(input_path)

# with open(output_path, 'w') as f:
#     for user in df['user'].unique():
#         items = df[df['user'] == user]['item'].to_list()
#         f.write('\t'.join(str(item) for item in items) + '\n')

In [19]:
# %%time
# reader = pd.read_table(input_path, chunksize=100000, sep=',')
# lst_to_remember = []
# prev_max_u = 0

# with open(output_path, 'w') as f:
#     for chunk in tqdm(reader):
#         curr_min_u = chunk['user'].min()
#         curr_max_u = chunk['user'].max()
    
#         for user in chunk['user'].unique():
#             items = lst_to_remember + chunk[chunk['user'] == user]['item'].to_list()
#             #print(items)
#             if user != curr_max_u:
#                 f.write('\t'.join(str(item) for item in items) + '\n')
#                 lst_to_remember = []
#             else:
#                 lst_to_remember = items
#         prev_max_u = curr_max_u

In [92]:
%%time
reader = pd.read_table(input_path, chunksize=100000, sep=',')
lst_to_remember = []
prev_max_u = -1

with open(output_path, 'w') as f:
    
    for chunk in reader:
        curr_min_u = chunk['user'].min()
        curr_max_u = chunk['user'].max()
    
        index, counts = np.unique(chunk['user'].values,  return_counts=True)
        tmp = np.split(chunk['item'].values,  np.cumsum(counts)[:-1])
        tmp = np.array(tmp)
        tmp = np.hstack((index.reshape((index.shape[0],  1)),
                 tmp.reshape((tmp.shape[0],  1)),
                ))
        
        if curr_min_u==prev_max_u:
            left = np.hstack((lst_to_remember, tmp[0][1]))
            tmp[0][1] = left
        else:
            f.write('\t'.join(str(item) for item in lst_to_remember) + '\n') 
         
        prev_max_u = tmp[-1][0]
        lst_to_remember = tmp[-1][1]
        
        for pair in tmp[:-1]:
            f.write('\t'.join(str(item) for item in pair[1]) + '\n') 
    
    f.write('\t'.join(str(item) for item in lst_to_remember) + '\n') 

CPU times: user 1min 33s, sys: 6.19 s, total: 1min 40s
Wall time: 1min 41s


Генератор для w2v, он считывает файл построчно, разделяет историю пользователя по \t на band_id и заменяет id на нормализованное название исполнителя

In [7]:
class TextToW2V:
    def __init__(self, file_path: str) -> None:
        """
        iterator for w2v out-of-memory training
        :param file_path: path to dataset in text format: one string for each user, only book IDs separated with ' '
        """
        self.file_path = file_path
        self.band_id_to_name = (pd.read_csv(meta_path, 
                                            sep='\t', usecols=['band_processed', 'band_id'],)
                                .set_index('band_id')
                                ['band_processed'].to_dict()
                               )

    def __iter__(self):
        """
        iterate over txt file and return user items
        :return:
        """
        for line in tqdm(open(self.file_path, 'r')):
            items = line.split('\t')
            res = [self.band_id_to_name.get(int(i_id)) for i_id in items]
            yield [x for x in res if x is not None]

### дефолтное дообучение w2v, можно поробовать разные гиперпараметры
вспомните статью с их тюнигом: https://arxiv.org/pdf/1804.04212.pdf

In [8]:
# !sed 1d $output_path > $output_path2

In [13]:
# corpus = TextToW2V(output_path2)
corpus = TextToW2V(w2v_dataset_path)

In [19]:
model = word2vec.Word2Vec(corpus, 
                          min_count=20, 
                          sg=1, 
                          iter=5, 
                          window=10, 
                          workers=multiprocessing.cpu_count(),
                         )

print(f'Модель выучила {len(model.wv.vocab)} исполнителей.')

50000it [00:04, 10172.84it/s]
50000it [00:20, 2440.50it/s]
50000it [00:19, 2576.62it/s]
50000it [00:20, 2461.57it/s]
50000it [00:26, 1900.20it/s]
50000it [00:22, 2242.05it/s]


Модель выучила 15685 исполнителей.


### Smoke test модели

In [20]:
model.wv.most_similar('nirvana')

[('metallica', 0.6461888551712036),
 ('lustra', 0.624983012676239),
 ('ac dc', 0.6226109266281128),
 ('foo fighters', 0.6181728839874268),
 ('pearl jam', 0.615071177482605),
 ('ramones', 0.6131055355072021),
 ('the misfits', 0.6067672967910767),
 ('slipknot', 0.6042327284812927),
 ('avenged sevenfold', 0.6010735034942627),
 ('mastodon', 0.6001443266868591)]

### После обучения модели:
- зайдите на свою страницу с музыкой вконтакте или я.музыки (подойдет сервис, где можно открыть свой плелист в вебе)
- выделите и скопируйте весь текст нас транице (Ctrl+A, Ctrl+C)
- после этого запустить ячейку ниже (никуда вставлять текст не надо, pandas заберет его из буфера обмена)

In [18]:
df = pd.read_clipboard(sep='no_such_sep')
words = df[df.columns[0]]

my_bands = [word 
            for word in words.str.lower() 
            if word in model.wv.vocab]

recommended =  model.wv.most_similar(positive=my_bands, topn=500)
recommended = [band 
               for band in recommended 
               if band not in my_bands
              ][:20]

for band, score in recommended:
    link = f'https://music.yandex.ru/search?text={band.replace(" ", "%20")}'
    print('\n', band, '\n', link)


 supernatural 
 https://music.yandex.ru/search?text=supernatural

 syd matters 
 https://music.yandex.ru/search?text=syd%20matters

 the national 
 https://music.yandex.ru/search?text=the%20national

 smokefishe 
 https://music.yandex.ru/search?text=smokefishe

 looptroop 
 https://music.yandex.ru/search?text=looptroop

 wolf alice 
 https://music.yandex.ru/search?text=wolf%20alice

 mono 
 https://music.yandex.ru/search?text=mono

 wye oak 
 https://music.yandex.ru/search?text=wye%20oak

 methods of mayhem 
 https://music.yandex.ru/search?text=methods%20of%20mayhem

 kodaline 
 https://music.yandex.ru/search?text=kodaline

 xxyyxx 
 https://music.yandex.ru/search?text=xxyyxx

 spice 
 https://music.yandex.ru/search?text=spice

 joe lynn turner 
 https://music.yandex.ru/search?text=joe%20lynn%20turner

 lil tracy 
 https://music.yandex.ru/search?text=lil%20tracy

 pierre van dormael 
 https://music.yandex.ru/search?text=pierre%20van%20dormael

 medusa scream 
 https://music.yandex.ru/